In [1]:
!nvidia-smi

Wed Mar  2 21:02:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 511.23       Driver Version: 511.23       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:07:00.0  On |                  N/A |
|  0%   55C    P0    45W / 215W |   1248MiB /  8192MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 1. 计算设备

In [2]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')
# 访问CPU；访问第0个GPU，访问第1个GPU

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

### 查询可用gpu的数量

In [3]:
torch.cuda.device_count()

1

###  这两个函数允许我们在不存在所需所有GPU的情况下运行代码
- 保证代码能运行

In [4]:
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

# 2. 在GPU上创建tensor
- 查询张量所在的设备

In [5]:
x = torch.tensor([1, 2, 3])
x.device    # 默认在CPU的内存上

device(type='cpu')

### 存储在GPU上

In [11]:
X = torch.ones(2, 3, device=try_gpu())
X, X.device

(tensor([[1., 1., 1.],
         [1., 1., 1.]], device='cuda:0'),
 device(type='cuda', index=0))

### 在第二个GPU上创建一个随机张量

In [10]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y, Y.device

(tensor([[0.4768, 0.2308, 0.8523],
         [0.6857, 0.5603, 0.7938]]),
 device(type='cpu'))

### 不同内存里数据的计算
- 保证性能：避免把GPU数据传到CPU，再做计算。
<img src="./pic/多GPU计算.PNG" width=500 height=500>

In [14]:
Z = Y.cuda(0)
print(Y)
print(Z)

tensor([[0.4768, 0.2308, 0.8523],
        [0.6857, 0.5603, 0.7938]])
tensor([[0.4768, 0.2308, 0.8523],
        [0.6857, 0.5603, 0.7938]], device='cuda:0')


In [15]:
X + Z

tensor([[1.4768, 1.2308, 1.8523],
        [1.6857, 1.5603, 1.7938]], device='cuda:0')

In [17]:
Z.cuda(0) is Z

True

# 3. 神经网络与GPU

In [19]:
net = nn.Sequential(nn.Linear(3, 1))

# .to挪到0号gpu上
net = net.to(device=try_gpu())

# X也在0号GPU上
net(X)

tensor([[-0.9879],
        [-0.9879]], device='cuda:0', grad_fn=<AddmmBackward>)

### 确认模型参数存储在同一个GPU上

In [20]:
net[0].weight.data.device

device(type='cuda', index=0)

# 总结

- 我们可以指定用于存储和计算的设备，例如CPU或GPU。默认情况下，数据在主内存中创建，然后使用CPU进行计算。

- 深度学习框架要求计算的所有输入数据都在`同一设备`上，无论是CPU还是GPU。

- 不经意地移动数据可能会显著降低性能。一个典型的错误如下：计算GPU上每个小批量的损失，并在命令行中将其报告给用户（或将其记录在NumPy ndarray中）时，将触发全局解释器锁，从而使所有GPU阻塞。最好是为GPU内部的日志分配内存，并且只移动较大的日志。